In [1]:
import glob 
import os
import cv2
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

Populating the interactive namespace from numpy and matplotlib


# post process:deleting extras

In [2]:
files = glob.glob('./out_ror_250st_342epo/*.png')
for item in files:
    img = cv2.imread(item, cv2.IMREAD_GRAYSCALE)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1
    if len(sizes)!= 0:
        target_size = sizes[-1]
    img2 = np.zeros((output.shape))
    for i in range(0, nb_components):
        if sizes[i] == target_size:
            img2[output == i + 1] = 255
    cv2.imwrite('./out_ror_250st_342epo_postpro/'+item.split('/')[-1],img2)

# evaluate after postprocess

In [3]:
from sklearn.metrics import confusion_matrix
total_miou = 0
total_dice = 0
total_fpr = 0
total_tpr = 0
# img_out = glob.glob('/home/sara/resnet50_segnet_5epoch/*.png')
img_out = glob.glob('./ror50_unet_500epo_postpro/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
total = 0
for pic in range(0,len(img_out)):
   
    m_out=cv2.imread(img_out[pic])
#     print(np.unique(m_out))
    prediction=np.reshape(np.where(m_out>0,1,0),(-1))
    target=np.reshape(cv2.imread(img_label[pic]),(-1))

    cm = confusion_matrix(target,prediction)
    tn = cm[0][0] #true positive
    fp = cm[0][1] #false positive
    fn = cm[1][0] #false negative
    tp = cm[1][1] #true positive

    total_miou = 0.5*((tn/(tn+fp+tn+fn-tn)) + (tp/(fn+tp+fp+tp-tp))) + total_miou
    total_dice = ((tn/(tn+fp+tn+fn)) + (tp/(fn+tp+fp+tp))) + total_dice
    tpr = (tp/(tp + fn)) * 100
    fpr = (fp/(fp + tn)) * 100
#     print('tpr: ',tpr,'\n')
#     print('fpr: ',fpr,'\n')
#     total_fpr = ((fp/(fp + tn)) * 100) + total_fpr
#     total_tpr = ((tp/(tp + fn)) * 100) + total_tpr
    total_fpr = fpr + total_fpr
    total_tpr = tpr + total_tpr
print('total_miou:',(total_miou/len(img_out)),'\n')
print('total_dice*',(total_dice/len(img_out)),'\n')
print('total_fpr:',(total_fpr/len(img_out)),'\n')
print('total_tpr:',(total_tpr/len(img_out)),'\n')

total_miou: 0.756566990998035 

total_dice* 0.7781519611182206 

total_fpr: 0.0561458030849376 

total_tpr: 54.68969999873165 



# Calculate the mean percent of missed pixel and extras pixles

In [3]:
total_miss = 0
total_extra = 0
l=0
img_out = glob.glob('./out_ror_250st_342epo_postpro/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
print(len(img_out))
for pic in range(0,len(img_out)):
    l=l+1
    miss_percent=0
    extra_pixels=0
    extra=0
    mean_erro=0
    equal=0
    #read images gt and output
    m_out=cv2.imread(img_out[pic])
    m_out=np.where(m_out>0,1,0)
    m_label=cv2.imread(img_label[pic])
    #check the result for missed and extras
    obj=np.unique(m_label, return_counts=True)[1][1] #target pixels of each image 
    background=np.unique(m_label, return_counts=True)[1][0] #background pixels of each image 
    result=np.unique(m_label-m_out,return_counts=True)
    if -1 in result[0]:
        index=np.where(result[0]==-1)
        extra_pixels=result[1][index]
        total_extra = extra_pixels + total_extra
        
    if 0 in result[0]:
        index=np.where(result[0]==0)
        equal=equal+1
        
    if 1 in result[0]:
        index=np.where(result[0]==1)
        miss_percent=result[1][index]/obj * 100
        total_miss = miss_percent + total_miss
        
print('total miss: ',total_miss/len(img_out))
print('total extra: ',np.int(total_extra/len(img_out)))

279
total miss:  [67.06315236]
total extra:  75
